In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml

data = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=True)

data

kolone = ['y']
kolone.extend('x'+str(i+1) for i in range(784))

print(kolone[:10])

data = pd.concat((data[1], data[0]), axis = 1)

data.columns = kolone

data
data.iloc[:, 0] = pd.to_numeric(data.iloc[:, 0])
data.iloc[:, 0] = np.where(data.iloc[:,0] == 5, 1, 0)
datatrain = data.sample(50000, random_state=42)
datadev = data.drop(index = datatrain.index).sample(10000, random_state=42)
datatest = data.drop(index = datatrain.index,).drop(index = datadev.index)
print(len(set(datatrain.index) | set(datadev.index) | set(datatest.index)))

['y', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9']


C:\Users\bogdan.sliskovic\AppData\Local\Temp\ipykernel_1000\2556689033.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        5
1        0
2        4
3        1
4        9
        ..
69995    2
69996    3
69997    4
69998    5
69999    6
Name: y, Length: 70000, dtype: int64' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  data.iloc[:, 0] = pd.to_numeric(data.iloc[:, 0])


70000


In [2]:
np.set_printoptions(suppress=True, precision=6)

In [3]:
data.y.unique()

array([1, 0])

In [ ]:
def feature(df):
    x = df.drop('y',axis = 1).values / 255
    m = x.shape[0]
    x = np.concatenate((np.ones((m,1)), x), axis = 1)
    return x

def target(data):
    y = data.iloc[:,0].values.reshape(-1,1)
    return y

xtrain, xdev, xtest = tuple(feature(skup) for skup in [datatrain, datadev, datatest])

ytrain, ydev, ytest = tuple(target(skup) for skup in [datatrain, datadev, datatest])

In [8]:
xtrain.max()

np.float64(1.0)

In [9]:
xtrain.shape, ytrain.shape, xdev.shape, ydev.shape, xtest.shape, ytest.shape

((50000, 785), (50000, 1), (10000, 785), (10000, 1), (10000, 785), (10000, 1))

In [ ]:
m,n = xtrain.shape

np.random.seed(42)
w = np.random.rand(n,1) - .5

z = xtrain @ w
pred = 1 / (1 + np.exp(-z))
pred

In [13]:
l = -(np.mean(ytrain * np.log(pred) + (1 - ytrain) * np.log(1 - pred)))
l

np.float64(0.7199538548303979)

In [14]:
gradijenti = (xtrain.T @ (pred - ytrain)) / m
gradijenti[:15]

array([[0.283753],
       [0.      ],
       [0.      ],
       [0.      ],
       [0.      ],
       [0.      ],
       [0.      ],
       [0.      ],
       [0.      ],
       [0.      ],
       [0.      ],
       [0.      ],
       [0.      ],
       [0.      ],
       [0.      ]])

In [15]:
gradijenti.sum()

np.float64(27.392207106705747)

In [16]:
w -= .1 * gradijenti

In [17]:
z = xtrain @ w
pred = 1 / (1 + np.exp(-z))
pred

array([[0.121145],
       [0.147565],
       [0.018718],
       ...,
       [0.166038],
       [0.165289],
       [0.147016]], shape=(50000, 1))

In [18]:
abs(z).max()

np.float64(9.887740495657864)

In [19]:
l = -(np.mean(ytrain * np.log(pred) + (1 - ytrain) * np.log(1 - pred)))
l

np.float64(0.5150179267013719)

In [20]:
pred[:10]

array([[0.121145],
       [0.147565],
       [0.018718],
       [0.104901],
       [0.097001],
       [0.051358],
       [0.015987],
       [0.342057],
       [0.477761],
       [0.230201]])

In [21]:
loss = []
for i in range (3000):  
    lr = 0.01
    z = np.clip(xtrain @ w, -500, 500)
    pred = 1 / (1 + np.exp(-z))
    l = -(np.mean(ytrain * np.log(pred+.00000001) + (1 - ytrain) * np.log(1 - pred+.00000001)))
    gradijenti = (xtrain.T @ (pred - ytrain)) / m
    w -= lr * gradijenti
    l2grad = np.linalg.norm(gradijenti)
    if i % 500 == 0:
        lr = lr *.5

    if i % 100 == 0:
        loss.append(l)
        print(f"Iteracija {i}, loss: {l}, velicina gradijenta{l2grad}")

    if l2grad < 0.005:
        break

Iteracija 0, loss: 0.5150178590786231, velicina gradijenta0.8618085200750397
Iteracija 100, loss: 0.36724235054210164, velicina gradijenta0.21515314794410034
Iteracija 200, loss: 0.32865985333862574, velicina gradijenta0.1810715465419959
Iteracija 300, loss: 0.3000315268427778, velicina gradijenta0.15789102349460796
Iteracija 400, loss: 0.2779587095750731, velicina gradijenta0.13969503216867554
Iteracija 500, loss: 0.26045138492903097, velicina gradijenta0.1252704991099562
Iteracija 600, loss: 0.24621195012873642, velicina gradijenta0.11363119239601499
Iteracija 700, loss: 0.23438477695484453, velicina gradijenta0.10405109056089391
Iteracija 800, loss: 0.2243909738150802, velicina gradijenta0.09601666855175857
Iteracija 900, loss: 0.21582669434075288, velicina gradijenta0.08916718584249353
Iteracija 1000, loss: 0.20840144660169402, velicina gradijenta0.083246726958684
Iteracija 1100, loss: 0.20190031416872906, velicina gradijenta0.07807029081274815
Iteracija 1200, loss: 0.1961603248079

In [29]:
z = np.clip(xtrain @ w,-500,500)
pred = 1 / (1 + np.exp(-z))


In [30]:
predtrain = (pred > 0.5).astype(int)

In [31]:
preciznost = np.mean(predtrain == ytrain)
print("Accuracy na trening skupu:", preciznost)

Accuracy na trening skupu: 0.94842


In [32]:
zdev = np.clip(xdev @ w, -500, 500)
pred = 1 / (1 + np.exp(-zdev))
predicted_classes = (pred >= 0.5).astype(int)

# Računaj tačnost poređenjem predviđenih klasa sa stvarnim oznakama
accuracy = np.mean(predicted_classes == ydev)
print("Accuracy na dev skupu:", accuracy)

Accuracy na dev skupu: 0.9496


In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [27]:
model = LogisticRegression(penalty = None,max_iter=1000)
model.fit(xtrain, ytrain.reshape(-1,))

ypredSklearn = model.predict(xtrain)

print(abs(ypredSklearn - predtrain.reshape(-1,)).sum())

print(accuracy_score(ypredSklearn, ytrain))

print(accuracy_score(predtrain, ytrain))

ypredSklearn.shape, pred.shape

2065
0.9782
0.94842


((50000,), (10000, 1))

In [28]:
print(accuracy_score(model.predict(xdev), ydev))

0.9754


SA REGULARIZACIJOM

In [33]:
np.random.seed(42)
w = np.random.rand(n,1) - .5

z = xtrain @ w
pred = 1 / (1 + np.exp(-z))
pred

array([[0.377692],
       [0.345481],
       [0.045587],
       ...,
       [0.273751],
       [0.377685],
       [0.338272]], shape=(50000, 1))

In [ ]:
l = -(np.mean(ytrain * np.log(pred) + (1 - ytrain) * np.log(1 - pred)))
l